In [79]:
from selenium import webdriver
import time
import datetime
import math
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import openpyxl
import pandas as pd
import warnings

In [80]:
def charms (i):
    name = str(df.iloc[i]['name'])
    if "крест" in name or "христиан" in name or "икона" in name or "мусульман" in name or "спаси" in name or "сохрани" in name or "месяц" in name or "аллах" in name or "пророк" in name or "будда" in name or "отче" in name or "молитва" in name or "икона" in name or "лунниц" in name or "ангел" in name or "эрцгамм" in name or "чудо" in name or "фатим" in name or "иисус" in name or "б.м" in name or "давид" in name or "матрон" in name or "госп" in name or "хамс" in name:
        return "КУЛЬТ"
    else:
        return "ДЕКОР"

In [81]:
def bracs (link):
    url =f'https://card.wb.ru/cards/v2/detail?appType=1&curr=rub&dest=-1257786&spp=30&nm={link}'
    sizes = requests.get(url, verify=False, timeout=30).json()
    try:
        r = sizes['data']['products'][0]['sizes'][0]['name']
        if r.rfind("-") > -1:
            wei = r[r.rfind("-")+1:].strip().lower().replace(",",".").replace("гр.","").replace("гр","")
        elif r.rfind("~") > -1:
            wei = r[r.rfind("~")+1:].strip().lower().replace(",", ".").replace("гр.","").replace("гр","")
        elif r.rfind(" ") > -1:
            wei = r[r.rfind(" ")+1:].strip().lower().replace(",", ".").replace("гр.","").replace("гр","")
        wei = float(wei)
        if wei > 0 and wei < 15:
            return wei
    except:
        return np.nan

In [82]:
def card (art):
    group = np.nan
    name = np.nan
    weight = np.nan
    lock_type = np.nan
    metal = np.nan
    insert = np.nan
    competitor = np.nan
    date = datetime.date.today()
    i = 0
    if int(art)//100000>spis[-1]:
        i = len(spis)
    else:
        while int(art)//100000 > spis[i]:
            i += 1
    i += 1
    if i < 10:
        i = "0"+str(i)
    current_link = f'https://basket-{i}.wbbasket.ru/vol{int(art)//100000}/part{int(art)//1000}/{art}/info/ru/card.json'
    try:
        basket = requests.get(current_link, verify=False, timeout=30).json()
    except:
        time.sleep(60)
        basket = requests.get(current_link, verify=False, timeout=30).json()
    try:
        competitor = basket['selling']['brand_name']
    except:
        pass
    try:
        name = basket['imt_name'].lower()
    except:
        pass
    try:
        group = basket['subj_name']
    except:
        pass
    try:
        for option in basket['options']:
            if option['name'] == 'Минимальный вес (г)':
                weight = float(option['value'].replace('г',''))
            elif option['name'] == 'Вставка':
                insert = option['value']
            elif option['name'] == 'Вид замка':
                lock_type = option['value']
            elif option['name'] == 'Состав ювелирного изделия':
                metal = option['value']
    except:
        pass
    try:
        for option in basket['grouped_options'][0]['options']:
            if option['name'] == 'Минимальный вес (г)':
                weight = float(option['value'].replace('г',''))
            elif option['name'] == 'Вставка':
                insert = option['value']
            elif option['name'] == 'Вид замка':
                lock_type = option['value']
            elif option['name'] == 'Состав ювелирного изделия':
                metal = option['value'].lower()
    except:
        pass
    try:
        if np.isnan(weight) and group in ["ЦБ БРАСЛЕТЫ","ЦБ ЦЕПИ"]:
            weight = float(bracs(art))
    except:
        pass
    try:
        if group == "ЦБ ЦЕПИ":
            for option in basket['options']:
                if 'плетен' in option['name']:
                    lock_type = option['value']
    except:
        pass
    try:
        if group == "ЦБ ЦЕПИ":
            for option in basket['grouped_options'][0]['options']:
                if 'плетен' in option['name']:
                    lock_type = option['value']
    except:
        pass
    return date, name, competitor, group, weight, insert, lock_type, metal

In [83]:
def tn (i):
    inss = str(df.iloc[i]['insert']).split(',')
    for ins in inss:
        ins = str(ins).strip().lower()
        insertgroup = ""
        if ins == "бриллиант" or ins == "сапфир" or ins =="топаз" or ins == "изумруд" or ins == "бриллиант натуральный" or ins == "сапфир натуральный" or ins == "топаз натуральный" or ins == "изумруд натуральный" or ins =="бриллианты" or ins == "":
            insertgroup = "ДК"
            tovnap  = 'ДК'
            break
        elif ins == "фианит" or ins == "бриллиант искусственный" or ins == "бриллиант выращенный" or ins =="бриллианит" or ins == "бриллиант синтетический" or "фианит" in str(df.iloc[i]['insert']).lower():
            insertgroup = "ИФ"
            tovnap = "ПДК"
        elif "нет" in ins or "без" in ins or  str(df.iloc[i]['insert']).lower().strip() == "золото" or ins == "nan":
            insertgroup = "БК"
            tovnap  = 'БК'
            break
        elif "бр" in ins and "искусственный" not in ins and "выращенный" not in ins and "синтетический" not in ins and "браслет" not in ins:
            insertgroup = "ДК"
            tovnap = 'ДК'
            break
        elif insertgroup != "ИФ":
            insertgroup = "ПДК"
            tovnap = "ПДК"
    return insertgroup, tovnap

In [92]:
married = True
parsesize = False
warnings.filterwarnings('ignore')
links=list()
spis = [143, 287, 431, 719, 1007, 1061, 1115, 1169, 1313, 1601, 1655, 1919, 2045, 2189, 2405]
p=1
arts = list()
prices = list()
date = list()
discount = list()

In [93]:
mwd = webdriver.Chrome()
mwd.maximize_window()
df =pd.DataFrame(columns=['date','name','competitor','group','weight','insert','lock_type','metal','price','price_old','discount','art','avg_price'])
while len(df.index)<150:
    url = f'https://www.wildberries.ru/catalog/0/search.aspx?page={p}&sort=popular&search=%D0%B7%D0%BE%D0%BB%D0%BE%D1%82%D0%BE&f1134=10592'
    mwd.get(url)
    time.sleep(1.5)
    step = 1900
    while True:
        step0=step-1900
        lh = mwd.execute_script("return document.body.scrollHeight")
        mwd.execute_script(f"window.scrollTo({step0}, {step});")
        time.sleep(1)
        nh = mwd.execute_script("return document.body.scrollHeight")
        step+=1900
        if nh==lh:
            break
    soup = bs(mwd.page_source, "html.parser")
    artss =  soup.find_all('article',id=True)
    for art in artss:
        artt = art['data-nm-id'] 
        try:
            rate = art.find_next('span',attrs={'class':'address-rate-mini address-rate-mini--sm'}).text
            prcount = art.find_next('span',attrs={'product-card__count'}).text
        except:
            rate = 1
            prcount = 1
        try:
            prcount = int(prcount[:prcount.find(" ")])
        except:
            prcount = 1
        if float(rate) >=4.7 and int(prcount)>=50:
            if str(art).find('price__lower-price wallet-price')>0:
                prl = art.findNext('ins',attrs={'class':'price__lower-price wallet-price'}).text.strip()[:-2].replace('\xa0','')
                po = math.ceil(int(prl)/0.95)
                discount = 0.95
            else:
                prl = art.findNext('ins',attrs={'class': 'price__lower-price'}).text.strip()[:-2]
                if prl.find('\xa0')>0:
                    prl = prl.replace('\xa0','')
                    po = prl
                    discount = 1
            params = card(artt)
            params = list(params)
            if not np.isnan(params[4]) and int(prl)/params[4]>3000 and "жемч" not in str(params[5]) and "кауч" not in str(params[5]) and "эмал" not in str(params[5]):  
                params.append(prl)
                params.append(po)
                params.append(discount)
                params.append(artt)
                params.append(int(prl)/params[4])
                df.loc[len(df.index)] = params
                if len(df.index) == 150:
                    break
    p+=1
df = df[['date','competitor','name','group','weight','price','price_old','discount','insert','metal','lock_type','art','avg_price']]
mwd.close()
df

,date,competitor,name,group,weight,price,price_old,discount,insert,metal,lock_type,art,avg_price
0,2024-03-21,Золотая Подкова,кольцо золотое 585 проба,Ювелирные кольца,0.95,6289,6620,0.95,фианит,золото красное,NaN,98618802,6620.000000
1,2024-03-21,Золотая Подкова,кольцо золотое 585 проба,Ювелирные кольца,1.30,9922,10445,0.95,фианит,золото красное; фианит,NaN,13294037,7632.307692
2,2024-03-21,DIALVI,кольцо золотое 585 пробы спаси и сохрани ювели...,Ювелирные кольца,0.92,8873,9340,0.95,фианит,золото,NaN,91721347,9644.565217
3,2024-03-21,Ювелирный Дом АРКАД,золотой браслет 585 на руку ювелирный адамас,Ювелирные браслеты,3.35,19260,19260,1.00,браслет золотой; Браслет на руку; золотой брас...,золото красное; красное золото,карабин,19133416,5749.253731
4,2024-03-21,DIALVI,колье на леске сердце,Ювелирные колье,1.25,6563,6909,0.95,фианит; леска,красное золото 585 пробы; фианит; леска,Шпрингель/тоггл; шпрингельный,91915422,5250.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2024-03-21,SOKOLOV,кольцо золотое 585 пробы с наношпинелью ювелирное,Ювелирные кольца,2.99,21162,21162,1.00,фианит; шпинель синтетическая,золото красное,NaN,8778771,7077.591973
146,2024-03-21,DIALVI,леска на шею красное золото,Ювелирные колье,0.35,2762,2908,0.95,без вставки,золото; золото красное,шпрингельный,91756959,7891.428571
147,2024-03-21,Ювелирные Традиции,подвеска на шею золотая 585 с топазом,Ювелирные подвески,0.29,2479,2610,0.95,топаз натуральный,золото красное,NaN,72778451,8548.275862
148,2024-03-21,Ювелирные Традиции,подвеска золото с топазом,Ювелирные подвески,0.40,3207,3376,0.95,топаз натуральный,золото красное,NaN,158358586,8017.500000


In [94]:
df['group'].value_counts()

group
Ювелирные кольца      57
Ювелирные подвески    41
Ювелирные серьги      31
Ювелирные браслеты    10
Ювелирные цепочки      7
Ювелирные колье        3
Ювелирные шнурки       1
Name: count, dtype: int64

In [95]:
tgs = list()
tnns = list()
for i in df.index:
    if df.iloc[i]['group'] == "Ювелирные подвески":
        tg = "ПОДВЕС " + charms(i)
    elif df.iloc[i]['group'] == "Ювелирные кольца":
        if "обруч" in df.iloc[i]['name'].lower() or "помолв" in df.iloc[i]['name'].lower() or "свад" in df.iloc[i]['name'].lower():
            tg = "КОЛЬЦА ОБРУЧ"
        else:
            tg = "КОЛЬЦА"
    elif df.iloc[i]['group'] == "Ювелирные цепочки":
        tg = "ЦБ ЦЕПИ"
    elif df.iloc[i]['group'] == "Ювелирные браслеты":
        tg = "ЦБ БРАСЛЕТЫ"
    elif df.iloc[i]['group']=="Ювелирные серьги":
        tg = "СЕРЬГИ"
    else:
        tg = ""
    if len(tg)>0 and "ЦБ" not in tg:
        tg = f'{tn(i)[0]} {tg}'.strip()
        tnn = tn(i)[1]
    elif "ЦБ" in tg:
        tnn = "ЦБ"
    else:
        tnn = "Другое"
        tg = "Другое"
    tgs.append(tg)
    tnns.append(tnn)
df.insert(loc=len(df.columns),column='product_group(ТГ)',value=tgs)
df.insert(loc=len(df.columns),column='product_direction(ТН)',value=tnns)

In [96]:
df

,date,competitor,name,group,weight,price,price_old,discount,insert,metal,lock_type,art,avg_price,product_group(ТГ),product_direction(ТН)
0,2024-03-21,Золотая Подкова,кольцо золотое 585 проба,Ювелирные кольца,0.95,6289,6620,0.95,фианит,золото красное,NaN,98618802,6620.000000,ИФ КОЛЬЦА,ПДК
1,2024-03-21,Золотая Подкова,кольцо золотое 585 проба,Ювелирные кольца,1.30,9922,10445,0.95,фианит,золото красное; фианит,NaN,13294037,7632.307692,ИФ КОЛЬЦА,ПДК
2,2024-03-21,DIALVI,кольцо золотое 585 пробы спаси и сохрани ювели...,Ювелирные кольца,0.92,8873,9340,0.95,фианит,золото,NaN,91721347,9644.565217,ИФ КОЛЬЦА,ПДК
3,2024-03-21,Ювелирный Дом АРКАД,золотой браслет 585 на руку ювелирный адамас,Ювелирные браслеты,3.35,19260,19260,1.00,браслет золотой; Браслет на руку; золотой брас...,золото красное; красное золото,карабин,19133416,5749.253731,ЦБ БРАСЛЕТЫ,ЦБ
4,2024-03-21,DIALVI,колье на леске сердце,Ювелирные колье,1.25,6563,6909,0.95,фианит; леска,красное золото 585 пробы; фианит; леска,Шпрингель/тоггл; шпрингельный,91915422,5250.400000,Другое,Другое
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2024-03-21,SOKOLOV,кольцо золотое 585 пробы с наношпинелью ювелирное,Ювелирные кольца,2.99,21162,21162,1.00,фианит; шпинель синтетическая,золото красное,NaN,8778771,7077.591973,ИФ КОЛЬЦА,ПДК
146,2024-03-21,DIALVI,леска на шею красное золото,Ювелирные колье,0.35,2762,2908,0.95,без вставки,золото; золото красное,шпрингельный,91756959,7891.428571,Другое,Другое
147,2024-03-21,Ювелирные Традиции,подвеска на шею золотая 585 с топазом,Ювелирные подвески,0.29,2479,2610,0.95,топаз натуральный,золото красное,NaN,72778451,8548.275862,ДК ПОДВЕС ДЕКОР,ДК
148,2024-03-21,Ювелирные Традиции,подвеска золото с топазом,Ювелирные подвески,0.40,3207,3376,0.95,топаз натуральный,золото красное,NaN,158358586,8017.500000,ДК ПОДВЕС ДЕКОР,ДК


In [97]:
df['product_group(ТГ)'].value_counts()

product_group(ТГ)
ИФ КОЛЬЦА           24
ИФ СЕРЬГИ           15
БК КОЛЬЦА ОБРУЧ     13
ЦБ БРАСЛЕТЫ         10
ДК КОЛЬЦА           10
БК ПОДВЕС ДЕКОР      8
БК ПОДВЕС КУЛЬТ      8
ДК ПОДВЕС ДЕКОР      8
ИФ ПОДВЕС ДЕКОР      7
ПДК ПОДВЕС ДЕКОР     7
ЦБ ЦЕПИ              7
БК СЕРЬГИ            7
БК КОЛЬЦА            5
ПДК СЕРЬГИ           5
Другое               4
ДК СЕРЬГИ            4
ПДК КОЛЬЦА           4
ИФ ПОДВЕС КУЛЬТ      3
ИФ КОЛЬЦА ОБРУЧ      1
Name: count, dtype: int64

In [98]:
df['product_direction(ТН)'].value_counts()

product_direction(ТН)
ПДК       66
БК        41
ДК        22
ЦБ        17
Другое     4
Name: count, dtype: int64

In [99]:
df.to_excel(f'wildberries(top)_{datetime.date.today().day}_{datetime.date.today().month}_{datetime.date.today().year}.xlsx',index=False)